In [83]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 24: Arithmetic Logic Unit ---</h2><p><a href="https://en.wikipedia.org/wiki/Magic_smoke" target="_blank">Magic smoke</a> starts leaking from the submarine's <a href="https://en.wikipedia.org/wiki/Arithmetic_logic_unit">arithmetic logic unit</a> (ALU). Without the ability to perform basic arithmetic and logic functions, the submarine can't produce cool patterns with its Christmas lights!</p>
<p>It also can't navigate. Or run the oxygen system.</p>
<p>Don't worry, though - you <em>probably</em> have enough oxygen left to give you enough time to build a new ALU.</p>
<p>The ALU is a four-dimensional processing unit: it has integer variables <code>w</code>, <code>x</code>, <code>y</code>, and <code>z</code>. These variables all start with the value <code>0</code>. The ALU also supports <em>six instructions</em>:</p>
<ul>
<li><code>inp a</code> - Read an input value and write it to variable <code>a</code>.</li>
<li><code>add a b</code> - Add the value of <code>a</code> to the value of <code>b</code>, then store the result in variable <code>a</code>.</li>
<li><code>mul a b</code> - Multiply the value of <code>a</code> by the value of <code>b</code>, then store the result in variable <code>a</code>.</li>
<li><code>div a b</code> - Divide the value of <code>a</code> by the value of <code>b</code>, truncate the result to an integer, then store the result in variable <code>a</code>. (Here, "truncate" means to round the value toward zero.)</li>
<li><code>mod a b</code> - Divide the value of <code>a</code> by the value of <code>b</code>, then store the <em>remainder</em> in variable <code>a</code>. (This is also called the <a href="https://en.wikipedia.org/wiki/Modulo_operation" target="_blank">modulo</a> operation.)</li>
<li><code>eql a b</code> - If the value of <code>a</code> and <code>b</code> are equal, then store the value <code>1</code> in variable <code>a</code>. Otherwise, store the value <code>0</code> in variable <code>a</code>.</li>
</ul>
<p>In all of these instructions, <code>a</code> and <code>b</code> are placeholders; <code>a</code> will always be the variable where the result of the operation is stored (one of <code>w</code>, <code>x</code>, <code>y</code>, or <code>z</code>), while <code>b</code> can be either a variable or a number. Numbers can be positive or negative, but will always be integers.</p>
<p>The ALU has no <em>jump</em> instructions; in an ALU program, every instruction is run exactly once in order from top to bottom. The program halts after the last instruction has finished executing.</p>
<p>(Program authors should be especially cautious; attempting to execute <code>div</code> with <code>b=0</code> or attempting to execute <code>mod</code> with <code>a&lt;0</code> or <code>b&lt;=0</code>  will cause the program to crash and might even <span title="Maybe this is what happened to the last one.">damage the ALU</span>. These operations are never intended in any serious ALU program.)</p>
<p>For example, here is an ALU program which takes an input number, negates it, and stores it in <code>x</code>:</p>
<pre><code>inp x
mul x -1
</code></pre>
<p>Here is an ALU program which takes two input numbers, then sets <code>z</code> to <code>1</code> if the second input number is three times larger than the first input number, or sets <code>z</code> to <code>0</code> otherwise:</p>
<pre><code>inp z
inp x
mul z 3
eql z x
</code></pre>
<p>Here is an ALU program which takes a non-negative integer as input, converts it into binary, and stores the lowest (1's) bit in <code>z</code>, the second-lowest (2's) bit in <code>y</code>, the third-lowest (4's) bit in <code>x</code>, and the fourth-lowest (8's) bit in <code>w</code>:</p>
<pre><code>inp w
add z w
mod z 2
div w 2
add y w
mod y 2
div w 2
add x w
mod x 2
div w 2
mod w 2
</code></pre>
<p>Once you have built a replacement ALU, you can install it in the submarine, which will immediately resume what it was doing when the ALU failed: validating the submarine's <em>model number</em>. To do this, the ALU will run the MOdel Number Automatic Detector program (MONAD, your puzzle input).</p>
<p>Submarine model numbers are always <em>fourteen-digit numbers</em> consisting only of digits <code>1</code> through <code>9</code>. The digit <code>0</code> <em>cannot</em> appear in a model number.</p>
<p>When MONAD checks a hypothetical fourteen-digit model number, it uses fourteen separate <code>inp</code> instructions, each expecting a <em>single digit</em> of the model number in order of most to least significant. (So, to check the model number <code>13579246899999</code>, you would give <code>1</code> to the first <code>inp</code> instruction, <code>3</code> to the second <code>inp</code> instruction, <code>5</code> to the third <code>inp</code> instruction, and so on.) This means that when operating MONAD, each input instruction should only ever be given an integer value of at least <code>1</code> and at most <code>9</code>.</p>
<p>Then, after MONAD has finished running all of its instructions, it will indicate that the model number was <em>valid</em> by leaving a <code>0</code> in variable <code>z</code>. However, if the model number was <em>invalid</em>, it will leave some other non-zero value in <code>z</code>.</p>
<p>MONAD imposes additional, mysterious restrictions on model numbers, and legend says the last copy of the MONAD documentation was eaten by a <a href="https://en.wikipedia.org/wiki/Japanese_raccoon_dog" target="_blank">tanuki</a>. You'll need to <em>figure out what MONAD does</em> some other way.</p>
<p>To enable as many submarine features as possible, find the largest valid fourteen-digit model number that contains no <code>0</code> digits. <em>What is the largest model number accepted by MONAD?</em></p>
</article>


In [84]:
from collections import deque
from pprint import pprint


type Instruction = list[tuple[str, str, str | int]]


@dataclass
class ALU:
    inputs: deque[int] = field(default_factory=lambda: deque([]))
    variables: dict[str, int] = field(default_factory=lambda: {c: 0 for c in "wxyz"})

    def inp(self, a: str) -> None:
        # inp a - Read an input value and write it to variable a.
        self.variables[a] = self.inputs.popleft()

    def add(self, a: str, b: str) -> None:
        # add a b - Add the value of a to the value of b, then store the result in variable a.
        self.variables[a] += self.variables[b] if isinstance(b, str) else b

    def mul(self, a: str, b: str) -> None:
        # mul a b - Multiply the value of a by the value of b, then store the result in variable a.
        self.variables[a] *= self.variables[b] if isinstance(b, str) else b

    def div(self, a: str, b: str) -> None:
        # div a b - Divide the value of a by the value of b,
        # truncate the result to an integer,
        # then store the result in variable a.
        # (Here, "truncate" means to round the value toward zero.)
        self.variables[a] //= self.variables[b] if isinstance(b, str) else b

    def mod(self, a: str, b: str) -> None:
        # mod a b - Divide the value of a by the value of b,
        # then store the remainder in variable a.
        #  (This is also called the modulo operation.)
        self.variables[a] %= self.variables[b] if isinstance(b, str) else b

    def eql(self, a: str, b: str) -> None:
        # eql a b - If the value of a and b are equal,
        # then store the value 1 in variable a.
        # Otherwise, store the value 0 in variable a.
        self.variables[a] = int(
            self.variables[a] == (self.variables[b] if isinstance(b, str) else b)
        )

    def load_and_run_instruction(self, instruction: Instruction) -> None:
        match instruction:
            case "inp", a:
                self.inp(a)
            case "add", a, b:
                self.add(a, b)
            case "mul", a, b:
                self.mul(a, b)
            case "div", a, b:
                self.div(a, b)
            case "mod", a, b:
                self.mod(a, b)
            case "eql", a, b:
                self.eql(a, b)
            case _:
                raise ValueError(f"Error {instruction=} does not exist")

    def reset(self) -> None:
        self.inputs = deque([])
        self.variables = {c: 0 for c in "wxyz"}


class Program:
    def __init__(self, program: str) -> None:
        self.program = self.parse_program(program)
        self.alu = ALU()

    @staticmethod
    def parse_program(program: str) -> list[Instruction]:
        is_integer = re.compile(r"-?[0-9]")
        return [
            tuple(int(e) if is_integer.match(e) else e for e in ins.strip().split())
            for ins in program.strip().splitlines()
        ]

    def run(self, inp: str | int) -> dict[str:int]:
        self.alu.inputs.extend([int(i) for i in str(inp)])

        for instruction in self.program:
            self.alu.load_and_run_instruction(instruction)

        return self.alu.variables

    def reset_ALU(self) -> None:
        self.alu.reset()

In [85]:
program_run_tests = [
    {
        "name": "Example 1",
        "program": """
            inp x
            mul x -1
        """,
        "inp": "1",
        "expected": {"w": 0, "x": -1, "y": 0, "z": 0},
    },
    {
        "name": "Example 2 a",
        "program": """
                inp z
                inp x
                mul z 3
                eql z x
        """,
        "inp": "11",
        "expected": {"w": 0, "x": 1, "y": 0, "z": 0},
    },
    {
        "name": "Example 2 a",
        "program": """
                inp z
                inp x
                mul z 3
                eql z x
        """,
        "inp": "26",
        "expected": {"w": 0, "x": 6, "y": 0, "z": 1},
    },
    {
        "name": "Example 3",
        "program": """
                inp w
                add z w
                mod z 2
                div w 2
                add y w
                mod y 2
                div w 2
                add x w
                mod x 2
                div w 2
                mod w 2
        """,
        "inp": 9,
        "expected": {
            "w": int(bin(9)[-4]),
            "x": int(bin(9)[-3]),
            "y": int(bin(9)[-2]),
            "z": int(bin(9)[-1]),
        },
    },
]


@test(tests=program_run_tests)
def program_run_test(program: str, inp: str | int) -> dict[str, int]:
    p = Program(program=program)
    return p.run(inp)


Test Example 1 passed, for program_run_test.
Test Example 2 a passed, for program_run_test.
Test Example 2 a passed, for program_run_test.
Test Example 3 passed, for program_run_test.
Success


In [86]:
program_consrutcor_tests = [
    {
        "name": "Example 1",
        "program": """
            inp x
            mul x -1
        """,
        "expected": [
            ("inp", "x"),
            ("mul", "x", -1),
        ],
    },
    {
        "name": "Example 2",
        "program": """
                inp z
                inp x
                mul z 3
                eql z x
        """,
        "expected": [
            ("inp", "z"),
            ("inp", "x"),
            ("mul", "z", 3),
            ("eql", "z", "x"),
        ],
    },
    {
        "name": "Example 3",
        "program": """
                inp w
                add z w
                mod z 2
                div w 2
                add y w
                mod y 2
                div w 2
                add x w
                mod x 2
                div w 2
                mod w 2
        """,
        "expected": [
            ("inp", "w"),
            ("add", "z", "w"),
            ("mod", "z", 2),
            ("div", "w", 2),
            ("add", "y", "w"),
            ("mod", "y", 2),
            ("div", "w", 2),
            ("add", "x", "w"),
            ("mod", "x", 2),
            ("div", "w", 2),
            ("mod", "w", 2),
        ],
    },
]


@test(tests=program_consrutcor_tests)
def program_consrutcor_test(program: str) -> list[Instruction]:
    p = Program(program=program)
    return p.program


Test Example 1 passed, for program_consrutcor_test.
Test Example 2 passed, for program_consrutcor_test.
Test Example 3 passed, for program_consrutcor_test.
Success


In [87]:
ALU_constructor_tests = [
    {
        "name": "Part I Test 1",
        "expected": {c: 0 for c in "wxyz"},
    },
]


@test(tests=ALU_constructor_tests)
def alu_constructor_test() -> dict[str, int]:
    alu = ALU()
    return alu.variables


Test Part I Test 1 passed, for alu_constructor_test.
Success


In [88]:
# from functools import reduce

# with open("../input/day24.txt") as f:
#     puzzle = f.read()

# MONAD = Program(puzzle)

# numbers = map(
#     lambda t: reduce(lambda v, e: e + 10 * v, t, 0),
#     product(range(9, 0, -1), repeat=14),
# )
# valid_model_number = -1
# for model_number in numbers:
#     results = MONAD.run(model_number)

#     if results["z"] == 0:
#         valid_model_number = model_number
#         break
#     MONAD.reset_ALU()


# print(f"Part I: { valid_model_number }")

In [ ]:
# https://trebledj.me/posts/aoc-2021-day-24/
# copied from above
from functools import reduce
import z3

file = "../input/day24.txt"

with open(file) as f:
    instrs = [l.strip().split() for l in f.read().splitlines()]

n = 14  # Number of digits.
block = 18

# Gather the "magic numbers" from the ALU program.
addx = []
addy = []
divz = []
for i in range(n):
    divz.append(int(instrs[i * block + 4][-1]))
    addx.append(int(instrs[i * block + 5][-1]))
    addy.append(int(instrs[i * block + 15][-1]))

# Make input ints.
inp = [z3.Int(f"inp_{x}") for x in range(n)]

# Create a z3 solver.
s = z3.Optimize()

# Constrain input to non-zero digits.
s.add(*[1 <= i for i in inp], *[i <= 9 for i in inp])

# Chain constraints. Each iteration will have a separate z variable.
# The constraints added will connect z[i+1] to z[i].
zs = [z3.Int(f"z_{i}") for i in range(n + 1)]
s.add(zs[0] == 0)
for i in range(n):
    x = inp[i] != (zs[i] % 26 + addx[i])
    s.add(
        zs[i + 1]
        == z3.If(x, (zs[i] / divz[i]) * 26 + inp[i] + addy[i], zs[i] / divz[i])
    )

s.add(zs[-1] == 0)  # Victory constraint.

# Construct full input to optimise (with place value).
full_inp = reduce(lambda acc, x: acc * 10 + x, inp, 0)


def get_inp(model):
    return "".join(str(model[i]) for i in inp)


# Part 1.
s.push()
s.maximize(full_inp)
s.check()
part1 = get_inp(s.model())
print("part1:", part1)
s.pop()

# Part 2.
s.push()
s.minimize(full_inp)
s.check()
part2 = get_inp(s.model())
print("part2:", part2)
s.pop()

part1: 79997391969649
part2: 16931171414113


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>79997391969649</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>As the submarine starts booting up things like the <a href="https://www.youtube.com/watch?v=RXJKdh1KZ0w" target="_blank">Retro Encabulator</a>, you realize that maybe you don't need all these submarine features after all.</p>
<p><em>What is the smallest model number accepted by MONAD?</em></p>
</article>


<link href="style.css" rel="stylesheet"></link>


<link href="style.css" rel="stylesheet"></link>
<main>
<article><p>That's the right answer!  You are <span class="day-success">one gold star</span> closer to finding the sleigh keys.</p><p>You have completed Day 24! You can <span class="share">[Share<span class="share-content">on
  <a href="https://bsky.app/intent/compose?text=I+just+completed+%22Amphipod%22+%2D+Day+23+%2D+Advent+of+Code+2021+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2021%2Fday%2F23" target="_blank">Bluesky</a>
  <a href="https://twitter.com/intent/tweet?text=I+just+completed+%22Amphipod%22+%2D+Day+23+%2D+Advent+of+Code+2021&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2021%2Fday%2F23&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I+just+completed+%22Amphipod%22+%2D+Day+23+%2D+Advent+of+Code+2021+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2021%2Fday%2F23';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this victory or <a href="/2021">[Return to Your Advent Calendar]</a>.</p></article>
</main>


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>16931171414113</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
